# Model Training

### Importing necessary packages and libraries

In [1]:
#Basic Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

### Import the dataset as Pandas DataFrame

In [2]:
df = pd.read_csv('data/vehicle_data_cleaned.csv')
data = df.copy()

In [3]:
df.head()

,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,seats,brand
0,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.40,1248.0,74,5.0,Maruti
1,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14,1498.0,103.52,5.0,Skoda
2,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.70,1497.0,78,5.0,Honda
3,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.00,1396.0,90,5.0,Hyundai
4,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.10,1298.0,88.2,5.0,Maruti


In [5]:
# Find unique values for categorical columns
categorical_columns = ['fuel', 'seller_type', 'transmission', 'owner', 'brand']
unique_values = {}
for column in categorical_columns:
    unique_values[column] = df[column].unique().tolist()

# Print the unique values
for column, values in unique_values.items():
    print(f"{column}: {values}")

fuel: ['Diesel', 'Petrol', 'LPG', 'CNG']
seller_type: ['Individual', 'Dealer', 'Trustmark Dealer']
transmission: ['Manual', 'Automatic']
owner: ['First Owner', 'Second Owner', 'Third Owner', 'Fourth & Above Owner', 'Test Drive Car']
brand: ['Maruti', 'Skoda', 'Honda', 'Hyundai', 'Toyota', 'Ford', 'Renault', 'Mahindra', 'Tata', 'Chevrolet', 'Fiat', 'Datsun', 'Jeep', 'Mercedes-Benz', 'Mitsubishi', 'Audi', 'Volkswagen', 'BMW', 'Nissan', 'Lexus', 'Jaguar', 'Land', 'MG', 'Volvo', 'Daewoo', 'Kia', 'Force', 'Ambassador', 'Ashok', 'Isuzu', 'Opel', 'Peugeot']


#### Prepare X and Y variables

In [4]:
X = df.drop(columns=['selling_price'],axis=1)

In [5]:
X.head()

,year,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,seats,brand
0,2014,145500,Diesel,Individual,Manual,First Owner,23.40,1248.0,74,5.0,Maruti
1,2014,120000,Diesel,Individual,Manual,Second Owner,21.14,1498.0,103.52,5.0,Skoda
2,2006,140000,Petrol,Individual,Manual,Third Owner,17.70,1497.0,78,5.0,Honda
3,2010,127000,Diesel,Individual,Manual,First Owner,23.00,1396.0,90,5.0,Hyundai
4,2007,120000,Petrol,Individual,Manual,First Owner,16.10,1298.0,88.2,5.0,Maruti


In [6]:
y = df['selling_price']

In [7]:
y

0       450000
1       370000
2       158000
3       225000
4       130000
         ...  
8123    320000
8124    135000
8125    382000
8126    290000
8127    290000
Name: selling_price, Length: 8128, dtype: int64

In [9]:
# Identify the categorical and numerical columns
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

In [10]:
print(num_features)
print(cat_features)

Index(['year', 'km_driven', 'mileage', 'engine', 'seats'], dtype='object')
Index(['fuel', 'seller_type', 'transmission', 'owner', 'max_power', 'brand'], dtype='object')


In [13]:
# Import Transformation Libraries
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer



In [14]:
numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)

In [15]:
X = preprocessor.fit_transform(X)

In [16]:
X.shape

(8128, 371)

In [18]:
# separate dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((6502, 371), (1626, 371))

##### Creating Evauation function for model training

In [19]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [20]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 189511.6776
- Mean Absolute Error: 95402.4165
- R2 Score: 0.9446
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 226292.8062
- Mean Absolute Error: 110489.6361
- R2 Score: 0.9219




C:\Users\Goodness\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 44780607054286.56, tolerance: 421708954444.7384
  model = cd_fast.sparse_enet_coordinate_descent(


Lasso
Model performance for Training set
- Root Mean Squared Error: 189517.5362
- Mean Absolute Error: 95487.2716
- R2 Score: 0.9446
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 227592.8692
- Mean Absolute Error: 110207.8750
- R2 Score: 0.9210


Ridge
Model performance for Training set
- Root Mean Squared Error: 210682.5046
- Mean Absolute Error: 108366.8901
- R2 Score: 0.9316
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 233748.9892
- Mean Absolute Error: 119195.9354
- R2 Score: 0.9166


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 177138.7975
- Mean Absolute Error: 67368.1207
- R2 Score: 0.9516
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 208949.2194
- Mean Absolute Error: 91545.6395
- R2 Score: 0.9334


Decision Tree
Model performance for Training set
- Root Mean Squared Error: 15133.1026
- Mean Absolute